### Section 1: Prediction

In [2]:
!pip install h2o

In [3]:
import pandas as pd
import h2o
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_squared_error

In [4]:
TOX = pd.read_excel("data.xlsx")

In [5]:
TOX.head()

,TAID,SMILES,Halogens,S,P,Other Metal,Hydroxyl or Ether,"Ketones, aldehydes, carboxylic acids, or esters",Primary Amine or Ammonium Ion,Secondary Amine or Aromatic Nitrogen,Tertiary Amine or Amide Nitrogen,Partially Aromatic,Aromatic,Aliphatic,mouse intraperitoneal LD50,group
0,TOX-145,Oc1ccc(O)cc1,0,0,0,0,1,-1,0,0,0,0,1,0,3.041835,3
1,TOX-245,S=P(N1CC1)(N1CC1)N1CC1,0,1,1,0,0,0,-1,-1,1,0,0,1,4.235584,5
2,TOX-1273,ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl,-1,0,0,0,0,0,0,0,0,0,0,1,3.366732,4
3,TOX-1279,CN(C)C(=N)N=C(N)N,0,0,0,0,0,0,1,1,1,0,0,0,2.641604,3
4,TOX-1282,Cc1ccc(C(C)C)cc1O,0,0,0,0,1,-1,0,0,0,0,1,0,3.311627,4


In [5]:
Xtrain = TOX.drop(columns=['TAID', 'SMILES', 'mouse intraperitoneal LD50', 'group'])
ytrain = TOX['mouse intraperitoneal LD50']

# Split the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:
# H2O.ai
h2o.init()

Xtrain_h2o = h2o.H2OFrame(X_train)
Xtest_h2o = h2o.H2OFrame(X_test)
X_valid_h2o = h2o.H2OFrame(X_valid)
ytrain_h2o = h2o.H2OFrame(list(y_train), column_names=["mouse intraperitoneal LD50"])
ytest_h2o = h2o.H2OFrame(list(y_test), column_names=["mouse intraperitoneal LD50"])
yvalid_h2o = h2o.H2OFrame(list(y_valid), column_names=["mouse intraperitoneal LD50"])

train_h2o = Xtrain_h2o.cbind(ytrain_h2o)
test_h2o = Xtest_h2o.cbind(ytest_h2o)
valid_h2o = X_valid_h2o.cbind(yvalid_h2o)

aml = H2OAutoML(max_runtime_secs=3600, max_models=10, seed=42, nfolds=5)
aml.train(y="mouse intraperitoneal LD50", training_frame=train_h2o, validation_frame=valid_h2o)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5rrcuzet
  JVM stdout: /tmp/tmp5rrcuzet/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5rrcuzet/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_j1ucs2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:07:50.67: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/10
# GBM base models (used / total),4/4
# XGBoost base models (used / total),2/3
# DRF base models (used / total),2/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [7]:
# View AutoML leaderboard
leaderboard = aml.leaderboard
print(leaderboard)

model_id                                                     rmse       mse       mae     rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231125_220749     0.621152  0.38583   0.442983  0.147663                  0.38583
StackedEnsemble_BestOfFamily_1_AutoML_1_20231125_220749  0.621268  0.385974  0.442914  0.147703                  0.385974
GBM_4_AutoML_1_20231125_220749                           0.622037  0.38693   0.443715  0.147868                  0.38693
GBM_3_AutoML_1_20231125_220749                           0.622823  0.387909  0.444732  0.148021                  0.387909
XGBoost_1_AutoML_1_20231125_220749                       0.623675  0.38897   0.44452   0.148334                  0.38897
GBM_2_AutoML_1_20231125_220749                           0.62397   0.389339  0.445799  0.148229                  0.389339
XGBoost_2_AutoML_1_20231125_220749                       0.625096  0.390745  0.444733  0.148659                  0.390745
DRF_1_AutoML_1_20231125_220

In [8]:
test_h2o = h2o.H2OFrame(X_test)
best_model = aml.leader

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
pred = best_model.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [10]:
import numpy as np

# y_test = y_test.to_numpy()
pred = pred.as_data_frame()
pred = pred.to_numpy()

In [11]:
mse = mean_squared_error(y_test, pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.37347685335216607


In [11]:
Xtrain = TOX.drop(columns=['TAID', 'SMILES', 'mouse intraperitoneal LD50', 'group'])
ytrain = TOX['mouse intraperitoneal LD50']

# Split the data into training (80%), and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=42)

In [ ]:
# One-hot coding
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
X_train = pd.get_dummies(X_train, columns=non_numeric_columns)
X_test = pd.get_dummies(X_test, columns=non_numeric_columns)
X_train, X_test = X_train.align(X_test, join='inner', axis=1)  # Align the DataFrames by the columns

In [ ]:
# Gradient Boosting Machine (GBM)
from sklearn.ensemble import GradientBoostingRegressor  # For regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

gbm_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm_regressor.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [ ]:
y_pred = gbm_regressor.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.4073268125724454


In [ ]:
# tuning

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

gbm = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.37976225299108496


In [ ]:
# XGBoost
import xgboost as xgb

xgb_regressor = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred = xgb_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.40768256461925156


In [ ]:
# tuning
xgb_model = xgb.XGBRegressor(random_state=42)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.38041239427871626


In [21]:
# Distributed Random Forest
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
y_pred = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.3731316375600981


In [ ]:
print(y_pred)

[3.50458322 2.93689589 3.09073575 ... 2.96503422 2.77806068 3.40790896]


In [ ]:
# tree.feature_importances_
rf = RandomForestRegressor(n_estimators=1000, min_samples_leaf=10,
                            max_features=20, random_state=42)
rf.fit(X_train, y_train)

feature_importances = rf.feature_importances_
feature_names = X_train.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

                                            Feature  Importance
8                  Tertiary Amine or Amide Nitrogen    0.132752
7              Secondary Amine or Aromatic Nitrogen    0.112487
10                                         Aromatic    0.110365
6                     Primary Amine or Ammonium Ion    0.099445
11                                        Aliphatic    0.096980
4                                 Hydroxyl or Ether    0.094074
5   Ketones, aldehydes, carboxylic acids, or esters    0.093801
1                                                 S    0.073839
0                                          Halogens    0.071170
2                                                 P    0.052840
9                                Partially Aromatic    0.048952
3                                       Other Metal    0.013295


### Section 2: Feature Relationship

correlation:

1.   S + Hydroxyl or Ether
2.   S + Ketones,aldehydes, carboxylic acids, or esters
3.   N + Ketones,aldehydes, carboxylic acids, or esters
4.   N + Hydroxyl or Ether
5.   Aromatic + hydroxyl or ether
6.   Aromatic + metal
7.   Hydroxyl or Ether + Ketones, aldehydes, carboxylic acids, or esters

Contingency Tables

In [6]:
from scipy.stats import chi2_contingency

In [7]:
chi2_contingency(pd.crosstab(TOX['S'], TOX['Hydroxyl or Ether']))

Chi2ContingencyResult(statistic=485.45632116882445, pvalue=3.841486007865234e-106, dof=2, expected_freq=array([[14373.68033089,  4170.60154679,  8557.71812233],
       [ 4347.31966911,  1261.39845321,  2588.28187767]]))

In [8]:
chi2_contingency(pd.crosstab(TOX['S'], TOX['Ketones, aldehydes, carboxylic acids, or esters']))

Chi2ContingencyResult(statistic=584.532626931176, pvalue=1.1758524155924075e-127, dof=2, expected_freq=array([[ 5163.34598714,  4170.60154679, 17768.05246608],
       [ 1561.65401286,  1261.39845321,  5373.94753392]]))

In [9]:
chi2_contingency(pd.crosstab(TOX['Primary Amine or Ammonium Ion'], TOX['Ketones, aldehydes, carboxylic acids, or esters']))

Chi2ContingencyResult(statistic=777.3921407959822, pvalue=6.0557441061831914e-167, dof=4, expected_freq=array([[ 4855.66375818,  3922.0766594 , 16709.25958242],
       [  817.501204  ,   660.3221621 ,  2813.1766339 ],
       [ 1051.83503782,   849.6011785 ,  3619.56378368]]))

In [12]:
correlation_matrix = Xtrain.corr()
spearman_corr_matrix = Xtrain.corr(method='spearman')

high_corr = correlation_matrix.abs() > 0.2
high_corr_matrix = correlation_matrix[high_corr]
print(high_corr_matrix)

                                                 Halogens    S    P  \
Halogens                                              1.0  NaN  NaN   
S                                                     NaN  1.0  NaN   
P                                                     NaN  NaN  1.0   
Other Metal                                           NaN  NaN  NaN   
Hydroxyl or Ether                                     NaN  NaN  NaN   
Ketones, aldehydes, carboxylic acids, or esters       NaN  NaN  NaN   
Primary Amine or Ammonium Ion                         NaN  NaN  NaN   
Secondary Amine or Aromatic Nitrogen                  NaN  NaN  NaN   
Tertiary Amine or Amide Nitrogen                      NaN  NaN  NaN   
Partially Aromatic                                    NaN  NaN  NaN   
Aromatic                                              NaN  NaN  NaN   
Aliphatic                                             NaN  NaN  NaN   

                                                 Other Metal  \
Halogens    

In [13]:
from sklearn.linear_model import LinearRegression
y = TOX['mouse intraperitoneal LD50']

In [14]:
X = TOX[['S', 'Hydroxyl or Ether']]

model = LinearRegression()
model.fit(X, y)
coefficients = model.coef_
intercept = model.intercept_
print(f'Intercept: {intercept}')
print(f'Coefficients: {coefficients}')

Intercept: 3.051724613608171
Coefficients: [-0.07674911  0.04122754]


In [15]:
X = TOX[['S', 'Ketones, aldehydes, carboxylic acids, or esters']]

model = LinearRegression()
model.fit(X, y)
coefficients = model.coef_
intercept = model.intercept_
print(f'Intercept: {intercept}')
print(f'Coefficients: {coefficients}')

Intercept: 3.0825300162447657
Coefficients: [-0.07227286 -0.08749409]


In [16]:
X = TOX[['Primary Amine or Ammonium Ion', 'Ketones, aldehydes, carboxylic acids, or esters']]

model = LinearRegression()
model.fit(X, y)
coefficients = model.coef_
intercept = model.intercept_
print(f'Intercept: {intercept}')
print(f'Coefficients: {coefficients}')

Intercept: 3.0593957383530377
Coefficients: [-0.01217715 -0.08864732]


In [17]:
X = TOX[['Secondary Amine or Aromatic Nitrogen', 'Ketones, aldehydes, carboxylic acids, or esters']]

model = LinearRegression()
model.fit(X, y)
coefficients = model.coef_
intercept = model.intercept_
print(f'Intercept: {intercept}')
print(f'Coefficients: {coefficients}')

Intercept: 3.0880304381343655
Coefficients: [-0.10337728 -0.07622773]


In [18]:
temp_TOX = TOX

In [22]:
temp_TOX['S_Hydroxyl_interaction'] = temp_TOX['S'] * temp_TOX['Hydroxyl or Ether']
X = temp_TOX[['S', 'Hydroxyl or Ether', 'S_Hydroxyl_interaction']]

rf = RandomForestRegressor()
rf.fit(X, y)

importances = rf.feature_importances_

for i, col in enumerate(X.columns):
    print(f"{col}: {importances[i]}")

S: 0.2795086700643132
Hydroxyl or Ether: 0.6408665751261673
S_Hydroxyl_interaction: 0.07962475480951955
